# Progression: Ordered Sequences for Workflows

**Progression** is an ordered sequence of UUIDs with Element identity. It combines:

- List-like operations: `append`, `extend`, `insert`, `remove`, `pop`
- Workflow operations: `move`, `swap`, `reverse`
- Idempotent set-like operations: `include`, `exclude`
- Query operations: `len`, `contains`, `getitem`, `index`

Progressions are primitives for state machine construction, representing execution order in workflow systems.

In [ ]:
# Setup
from uuid import uuid4

from lionherd_core.base import Element, Progression

## 1. Construction & Append/Extend

Create progressions with UUIDs or Elements, then add items using `append` or `extend`.

In [ ]:
# Empty progression
prog = Progression(name="task_queue")
print(f"Empty: {prog}")

# With UUIDs
tasks = [uuid4() for _ in range(3)]
prog_tasks = Progression(order=tasks, name="pending")
print(f"\nWith UUIDs: {prog_tasks}")

# With Elements (auto-converts to UUIDs)
elements = [Element() for _ in range(2)]
prog_elements = Progression(order=elements, name="active")
print(f"With Elements: {prog_elements}")

# Append and extend
prog.append(uuid4())
prog.extend([uuid4(), uuid4()])
print(f"\nAfter append/extend: {prog}, len={len(prog)}")

## 2. Query Operations

Check length, membership, and access items by index.

In [ ]:
uids = [uuid4() for _ in range(5)]
prog = Progression(order=uids, name="execution_order")

# Length and membership
print(f"Length: {len(prog)}")
print(f"First UUID in progression: {uids[0] in prog}")
print(f"Random UUID in progression: {uuid4() in prog}")

# Index access (supports negative indices)
print(f"\nFirst item: {prog[0]}")
print(f"Last item: {prog[-1]}")
print(f"Slice [1:3]: {prog[1:3]}")

# Find index
print(f"\nIndex of third UUID: {prog.index(uids[2])}")

# Iteration
print(f"\nAll items: {list(prog)}")

## 3. Workflow Operations: Move & Swap

Reorder items for priority scheduling or state transitions.

In [ ]:
# Create progression representing task priority
task1, task2, task3, task4 = uuid4(), uuid4(), uuid4(), uuid4()
prog = Progression(order=[task1, task2, task3, task4], name="priority_queue")

print(f"Initial order: {[str(u)[:8] for u in prog.order]}")

# Move task3 (index 2) to front (index 0) - high priority
prog.move(2, 0)
print(f"After move(2, 0): {[str(u)[:8] for u in prog.order]}")

# Swap first and last tasks
prog.swap(0, -1)
print(f"After swap(0, -1): {[str(u)[:8] for u in prog.order]}")

# Reverse entire queue
prog.reverse()
print(f"After reverse(): {[str(u)[:8] for u in prog.order]}")

## 4. Idempotent Operations: Include & Exclude

Set-like operations that are safe for retries and concurrent calls.

In [ ]:
prog = Progression(name="task_registry")
task_id = uuid4()

# include: add if not present (idempotent)
print(f"First include: {prog.include(task_id)}")
print(f"Second include (duplicate): {prog.include(task_id)}")
print(f"Length after two includes: {len(prog)}")

# exclude: remove if present (idempotent)
print(f"\nFirst exclude: {prog.exclude(task_id)}")
print(f"Second exclude (absent): {prog.exclude(task_id)}")
print(f"Length after two excludes: {len(prog)}")

# Contrast with append (NOT idempotent)
prog.append(task_id)
prog.append(task_id)  # Creates duplicate
print(f"\nAfter two appends: len={len(prog)} (allows duplicates)")

## 5. List Operations: Insert, Remove, Pop

Standard list operations with Element/UUID support.

In [ ]:
uid1, uid2, uid3 = uuid4(), uuid4(), uuid4()
prog = Progression(order=[uid1, uid3], name="tasks")

# Insert at position
prog.insert(1, uid2)
print(f"After insert(1, uid2): {[str(u)[:8] for u in prog.order]}")

# Remove specific item
prog.remove(uid2)
print(f"After remove(uid2): {[str(u)[:8] for u in prog.order]}")

# Pop last item
popped = prog.pop()
print(f"Popped last: {str(popped)[:8]}")
print(f"Remaining: {[str(u)[:8] for u in prog.order]}")

# Pop first item (queue behavior)
prog.extend([uuid4(), uuid4()])
first = prog.popleft()
print(f"\nPopped first: {str(first)[:8]}")
print(f"After popleft: {len(prog)} items remaining")

## 6. Serialization

Progressions inherit Element serialization with JSON support.

In [ ]:
# Create progression
uids = [uuid4() for _ in range(3)]
original = Progression(order=uids, name="workflow_steps")

# Serialize to dict (JSON mode - UUIDs as strings)
data = original.to_dict(mode="json")
print("Serialized fields:")
print(f"  name: {data['name']}")
print(f"  order (first UUID): {data['order'][0][:8]}...")
print(f"  id: {data['id'][:8]}...")

# Deserialize (roundtrip)
restored = Progression.from_dict(data)
print(f"\nRestored: {restored}")
print(f"Order preserved: {restored.order == original.order}")
print(f"ID preserved: {restored.id == original.id}")

## 7. Workflow State Machine Example

Progressions enable state machines for task execution tracking.

In [ ]:
# State machine with three progressions
pending = Progression(name="pending")
active = Progression(name="active")
completed = Progression(name="completed")

# Initialize with tasks
tasks = [uuid4() for _ in range(5)]
pending.extend(tasks)

print(f"Initial state - pending: {len(pending)}, active: {len(active)}, completed: {len(completed)}")

# Start first task (pending → active)
task = pending.popleft()
active.append(task)
print(f"After start - pending: {len(pending)}, active: {len(active)}, completed: {len(completed)}")

# Complete task (active → completed)
task = active.pop()
completed.append(task)
print(f"After complete - pending: {len(pending)}, active: {len(active)}, completed: {len(completed)}")

# Retry failed task (active → pending)
pending.extend(active.order)
active.clear()
print(f"After retry - pending: {len(pending)}, active: {len(active)}, completed: {len(completed)}")

print(f"\nCompleted order preserves execution history: {[str(u)[:8] for u in completed.order]}")

## Summary

**Progression** provides ordered sequence management for workflows:

**Core Operations:**
- Construction: `Progression(order=[...], name="...")`
- List ops: `append`, `extend`, `insert`, `remove`, `pop`, `popleft`, `clear`
- Queries: `len`, `in`, `[index]`, `index()`, iteration

**Workflow Operations:**
- Reordering: `move(from, to)`, `swap(i, j)`, `reverse()`
- Idempotent: `include(item)`, `exclude(item)`

**Key Properties:**
- Element inheritance (id, created_at, metadata)
- UUID or Element input (auto-converts)
- Duplicates allowed (list semantics)
- Serialization support (JSON roundtrip)
- Workflow primitives for state machines

**Use Cases:**
- Task queues and execution order
- State machine construction (pending/active/completed)
- Priority scheduling with move/swap
- Idempotent registration with include/exclude